[![author](https://img.shields.io/badge/author-titocampos-red.svg)](https://github.com/titocampos/data_science_na_pratica)![contributions welcome](https://img.shields.io/badge/contributions-welcome-brightgreen.svg?style=flat)

<p align="center">
  <img src="banner.png" >
</p>